<a href="https://colab.research.google.com/github/talitaburbulhan/raspagem-de-noticias-do-the-guardian-NIGERIA/blob/main/Pensamento_Computacional_Trabalho_Final_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo do programa: **extrair o título e o link das matérias** do site do The Guardian que faz a cobertura da Nigéria: 

🌐 https://guardian.ng/category/news/nigeria/

1️⃣ MANIPUALAÇÃO DE STRING

No HTML original tenho acesso às 18 matérias da primeira página. Com a técnica da manipulação de strings foi possível extraí-las:

In [ ]:
import requests
url = "https://guardian.ng/category/news/nigeria/"
resposta = requests.get(url)
html = resposta.text

partes = html.split('<span class="title" data-field="title"><a href="')
for parte in partes:
  subpartes = parte.split("</div>")
  conteudo = subpartes[0]
  if "</a></span>" not in conteudo:
    continue
  linha = conteudo.strip()
  registro = linha.split("</a></span>")
  link_e_titulo_juntos = registro[0]
  link, titulo = link_e_titulo_juntos.split('" data-field="title-link">')
  print(link, titulo) 

👍🏼 O código funcionou! 

👎🏼 Só conseguiu pegar os links e títulos da primeira página. Para conseguir mais resultados tive que usar outra técnica.

2️⃣ ARQUIVO JSON:

Ao inspecionar o site, pela aba Network, e clicar no botão "LOAD MORE" uma das requisições é um arquivo JSON. 

Para conseguir extrair links e títulos além dos que estavam na primeira página, foi fundamental analisar a estrutura da URL:
https://guardian.ng/wp-json/wp/v2/posts?offset=12&per_page=12&orderby=date&order=d[…]776%2C1553760%2C1553292%2C1553300%2C1553286%2C1553315%2C1553517  

Ela possui um **OFFSET=12** e um **PER_PAGE=12**. Álvaro me orientou a mudar os número desses parâmetros. Coloquei OFFSET=0 E PER_PAGE=100, dessa maneira consegui extrair as 100 últimas matérias do site.


In [ ]:
import requests
url = "https://guardian.ng/wp-json/wp/v2/posts?offset=0&per_page=100&orderby=date&order=desc&categories=46751%2C4%2C3%2C46753&exclude=1555218%2C1554939%2C1555082%2C1555031%2C1555065%2C1555075%2C1555085%2C1555099%2C1555064%2C1554965%2C1554948%2C1554938%2C1554846"
resposta = requests.get(url)
dados = (resposta.json()) 

for noticia in dados:                                                   
  materia, link = (noticia["title"]["rendered"], noticia["link"])      ## Esse JSON é uma lista e cada item da lista era um dicionário. 
  print(materia, link)

👍🏼 O código funcionouou e eu tive mais registros.

👎🏼 Só conseguiu pegar as 100 últimas matérias. E se eu quisesse ter acesso a conteúdos mais antigos?

Criei uma função chamada `extrai_dados` que tem os seguintes parâmetros: "offset" e "per_page". Dessa maneira, posso chamar a função diversas vezes, sendo que a cada vez, posso conseguir até 100 novos links. No código abaixo, a função foi chamada 3 vezes. O resultado foi um arquivo .CSV com os 300 últimos links e títulos de matérias postadas no site. 

👇🏼 Código final, com o trecho que exporta para CSV

In [ ]:
def extrai_dados(offset, per_page):
  import requests
  url = f"https://guardian.ng/wp-json/wp/v2/posts?offset={offset}&per_page={per_page}&orderby=date&order=desc&categories=46751%2C4%2C3%2C46753&exclude=1555218%2C1554939%2C1555082%2C1555031%2C1555065%2C1555075%2C1555085%2C1555099%2C1555064%2C1554965%2C1554948%2C1554938%2C1554846"
  resposta = requests.get(url)
  dados = (resposta.json()) 
  lista_materias = []
  for noticia in dados:                                                   
    materia, link = (noticia["title"]["rendered"], noticia["link"])
    dicio = {"materia": materia, "link":link}  
    lista_materias.append(dicio)
  return lista_materias

import csv
arquivo = open("materias-nigeria-7.csv", mode="w")
escritor = csv.DictWriter(arquivo, fieldnames = ["materia", "link"] )
escritor.writeheader()
primeiro_bloco = extrai_dados(0,100)
segunto_bloco = extrai_dados(100,100)
terceiro_bloco = extrai_dados(200,100)
blocos = primeiro_bloco + segundo_bloco + terceiro_bloco 
for resultado in blocos:
  escritor.writerow(resultado) 
arquivo.close()